In [28]:
#import necessary libraries
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt 

import pandas as pd  
import seaborn as sns 

# import data
data = pd.read_csv("datasets-35847-51854-Churn Modeling.csv")
data.head()


print(data)
#defining functions 
def calc_error_r(classi, X_train, X_test, y_train, y_test):
    classi.fit(X_train, y_train)
    y_test_predict = classi.predict(X_test)
    print("r square: ", format(r2_score(y_test, y_test_predict)))
    print("squared loss: ", format(mean_squared_error(y_test, y_test_predict)))

def calc_score(classi, X_train, X_test, y_train, y_test):
    classi.fit(X_train, y_train)
    print(classi.score(X_test, y_test))
    

def Average(lst): 
    return sum(lst) / len(lst) 
    

      RowNumber  CustomerId         Surname  CreditScore Geography  Gender  \
0             1    15634602        Hargrave          619    France  Female   
1             2    15647311            Hill          608     Spain  Female   
2             3    15619304            Onio          502    France  Female   
3             4    15701354            Boni          699    France  Female   
4             5    15737888        Mitchell          850     Spain  Female   
5             6    15574012             Chu          645     Spain    Male   
6             7    15592531        Bartlett          822    France    Male   
7             8    15656148          Obinna          376   Germany  Female   
8             9    15792365              He          501    France    Male   
9            10    15592389              H?          684    France    Male   
10           11    15767821          Bearce          528    France    Male   
11           12    15737173         Andrews          497     Spa

In [29]:
# preprocessing (apply any of imputation, scaling, feature selection, one hot encoding etc. as you feel necessary) 

#check if there are any missing values in the data 
data.isnull().sum()

#change string values to int## Male = 0, Female = 1, Germany = 0, Spain = 1, France = 2
data["Gender"].replace({"Male": 0, "Female": 1}, inplace=True)
data["Geography"].replace({"Germany": 0, "Spain": 1, "France": 2}, inplace=True)


#look for correlations between the feature vectors
#the Spearman correlation does not assume that both datasets are normally distributed.
from scipy import stats
print(stats.spearmanr(data["CreditScore"], data["Gender"]))
print(stats.spearmanr(data["CreditScore"], data["Age"]))
print(stats.spearmanr(data["CreditScore"], data["Tenure"]))
print(stats.spearmanr(data["CreditScore"], data["Balance"]))
print(stats.spearmanr(data["CreditScore"], data["NumOfProducts"]))
print(stats.spearmanr(data["CreditScore"], data["HasCrCard"]))
print(stats.spearmanr(data["CreditScore"], data["IsActiveMember"]))
print(stats.spearmanr(data["CreditScore"], data["EstimatedSalary"]))
print(stats.spearmanr(data["CreditScore"], data["Exited"]))
print(stats.spearmanr(data["IsActiveMember"], data["Exited"]))
print(stats.spearmanr(data["Balance"], data["EstimatedSalary"]))


#Prepare the data for testing and training set for #Exited
X_Exited = data.iloc[:, [3,4,5,6,7,8,9,10,11,12]]

#PCA ##################################

from sklearn.decomposition import PCA
pca = PCA(n_components=4)
principalComponents = pca.fit_transform(X_Exited)
principalDf = pd.DataFrame(data = principalComponents, columns = ['principal component 1', 'principal component 2', 'principal component 3', 'principal component 4'])
print(principalDf)

#delete 0 values
#X.query('line_race != 0')

#last collumn 
y_Exited = data.iloc[:,-1].values

# Splitting the data into the Training set and Test set  
from sklearn.model_selection import train_test_split
X_train_Exited, X_test_Exited, y_train_Exited, y_test_Exited = train_test_split(X_Exited, y_Exited, test_size=0.2, random_state=10)

#############################Prepare the data for testing and training set for #IsActiveMember##############################
X_Active = data.iloc[:, [3,6,7,8,9,10,11,13]]
#delete 0 values
#X.query('line_race != 0')
y_Active = data.iloc[:,-3].values

# Splitting the data into the Training set and Test set  
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import Ridge
lasso = linear_model.Lasso()

X_train_Active, X_test_Active, y_train_Active, y_test_Active = train_test_split(X_Active, y_Active, test_size=0.2, random_state=10)
#X_train_Active, X_test_Active, y_train_Active, y_test_Active = cross_validate(ridge, X_Active, y_Active, cv=5)


SpearmanrResult(correlation=0.0030114427900058186, pvalue=0.7633326620455019)
SpearmanrResult(correlation=-0.007974044311824222, pvalue=0.4252667030635392)
SpearmanrResult(correlation=0.001133174187231753, pvalue=0.9097901093099146)
SpearmanrResult(correlation=0.005686570567648804, pvalue=0.5696340276861511)
SpearmanrResult(correlation=0.012567727105426257, pvalue=0.20887488379726135)
SpearmanrResult(correlation=-0.003801819664491181, pvalue=0.7038446015276565)
SpearmanrResult(correlation=0.024262340713124095, pvalue=0.015254163694324864)
SpearmanrResult(correlation=0.001236524378517163, pvalue=0.9016026735590865)
SpearmanrResult(correlation=-0.023289396606662873, pvalue=0.019860952565325888)
SpearmanrResult(correlation=-0.1561282781889051, pvalue=1.3482685164858008e-55)
SpearmanrResult(correlation=0.011778003481770119, pvalue=0.2389186359196331)
      principal component 1  principal component 2  principal component 3  \
0             -76157.145982           -7194.761639              

In [30]:
#import important libs
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

# imputation Exited
X_train_Exited = imputer.fit_transform(X_train_Exited)
X_test_Exited = imputer.transform(X_test_Exited)

# imputation Active
X_train_Active = imputer.fit_transform(X_train_Active)
X_test_Active = imputer.transform(X_test_Active)

In [31]:
#import libs
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
rc = RobustScaler()

# scaling Exited 
X_train_Exited = rc.fit_transform(X_train_Exited)
X_test_Exited = rc.transform(X_test_Exited)
# scaling Active
X_train_Active = rc.fit_transform(X_train_Active)
X_test_Active = rc.transform(X_test_Active)

In [32]:
#defining the models that are needed for the function
#logistic regression ###########################################################################################
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score
clf_lg = LogisticRegression(random_state=0, multi_class='auto')#.fit(X_train_Exited, y_train_Exited)

#svm#####################################################################################################################
from sklearn import svm
clf_svm = svm.SVC()
clf_svm.fit(X_train_Exited, y_train_Exited)
#ridge regression######################################################
#We use ridge regression in order in order to shrink features who are not important in a bigger way so that features
#who are important have a bigger impact on the end result
from sklearn.linear_model import Ridge
clf_ridge = Ridge(alpha=1.0)
clf_ridge.fit(X_train_Exited, y_train_Exited)

C:\Users\Eiden\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [33]:
#logistic regression##################################################
calc_error_r(clf_lg, X_train_Exited, X_test_Exited, y_train_Exited, y_test_Exited)

#svm#################################################################
calc_error_r(clf_svm, X_train_Exited, X_test_Exited, y_train_Exited, y_test_Exited)

#ridge regression#####################################################
calc_error_r(clf_ridge, X_train_Exited, X_test_Exited, y_train_Exited, y_test_Exited)



C:\Users\Eiden\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Eiden\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


r square:  -0.18633581412670663
squared loss:  0.1975
r square:  0.08396854858570757
squared loss:  0.1525
r square:  0.13909619471885648
squared loss:  0.14332240459939952


In [34]:
#######faster way##############
##############using cross validation#################################
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
scores_lr = []
scores_svm = []
scores_rf = []

print(cross_val_score(clf_ridge, X_train_Exited, y_train_Exited))
print(cross_val_score(clf_svm, X_train_Exited, y_train_Exited))
print(cross_val_score(RandomForestClassifier(n_estimators=40), X_train_Exited, y_train_Exited))

#or#######################without using cross validation####################################
print(calc_score(clf_ridge, X_train_Exited, X_test_Exited, y_train_Exited, y_test_Exited))
print(calc_score(clf_svm, X_train_Exited, X_test_Exited, y_train_Exited, y_test_Exited))
print(calc_score(RandomForestClassifier(n_estimators=40), X_train_Exited, X_test_Exited, y_train_Exited, y_test_Exited))

[0.12788273 0.16592164 0.13979245]


C:\Users\Eiden\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\Eiden\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\Eiden\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Eiden\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 's

[0.85682159 0.85483871 0.85783946]


C:\Users\Eiden\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


[0.86169415 0.86196549 0.8555889 ]
0.13909619471885648
None


C:\Users\Eiden\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.8475
None
0.855
None
